# State estimation


In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [2]:
from pathlib import Path

import sympy as sp
import numpy as np
import xarray as xr
import xarray.ufuncs as xf
import matplotlib.pyplot as plt

from system_identification.iekf import IteratedExtendedKalmanFilter
from system_identification.load_assignment_data import load_assignment_data

In [3]:
u, v, w, cau = sp.symbols("u, v, w, C_alpha_u")
alpha_m, beta_m, v_m = sp.symbols("alpha_m, beta_m, V_m")
u_dot, v_dot, w_dot = sp.symbols("udot, vdot, wdot")

In [4]:
iekf = IteratedExtendedKalmanFilter(
    x=[u, v, w, cau],
    z=[alpha_m, beta_m, v_m],
    u=[u_dot, v_dot, w_dot],
    f=sp.Matrix([
        u_dot,
        v_dot,
        w_dot,
        0.
    ]),
    h=sp.Matrix([
        sp.atan(w / u) * (1 + cau),
        sp.atan(v / sp.sqrt(u**2 + w**2)),
        sp.sqrt(u**2 + v**2 + w**2)
    ]),
    g=sp.Matrix(np.diag([1, 1, 1, 0])),
    max_iterations=10,
    eps=1e-10,
)
iekf

$$x = \left( u, \  v, \  w, \  C_{\alpha u}\right)$$  
$$z = \left( \alpha_{m}, \  \beta_{m}, \  V_{m}\right)$$  
$$u = \left( \dot{u}, \  \dot{v}, \  \dot{w}\right)$$  
$$f(\dots) = \left[\begin{matrix}\dot{u}\\\dot{v}\\\dot{w}\\0.0\end{matrix}\right]$$  
$$g(\dots) = \left[\begin{matrix}1 & 0 & 0 & 0\\0 & 1 & 0 & 0\\0 & 0 & 1 & 0\\0 & 0 & 0 & 0\end{matrix}\right]$$  
$$h(\dots) = \left[\begin{matrix}\left(C_{\alpha u} + 1\right) \operatorname{atan}{\left(\frac{w}{u} \right)}\\\operatorname{atan}{\left(\frac{v}{\sqrt{u^{2} + w^{2}}} \right)}\\\sqrt{u^{2} + v^{2} + w^{2}}\end{matrix}\right]$$  
$$F_x(\dots) = \left[\begin{matrix}0 & 0 & 0 & 0\\0 & 0 & 0 & 0\\0 & 0 & 0 & 0\\0 & 0 & 0 & 0\end{matrix}\right]$$  
$$H_x(\dots) = \left[\begin{matrix}- \frac{w \left(C_{\alpha u} + 1\right)}{u^{2} \left(1 + \frac{w^{2}}{u^{2}}\right)} & 0 & \frac{C_{\alpha u} + 1}{u \left(1 + \frac{w^{2}}{u^{2}}\right)} & \operatorname{atan}{\left(\frac{w}{u} \right)}\\- \frac{u v}{\left(u^{2} + w^{2}\right)^{\frac{3}{2}} \left(\frac{v^{2}}{u^{2} + w^{2}} + 1\right)} & \frac{1}{\sqrt{u^{2} + w^{2}} \left(\frac{v^{2}}{u^{2} + w^{2}} + 1\right)} & - \frac{v w}{\left(u^{2} + w^{2}\right)^{\frac{3}{2}} \left(\frac{v^{2}}{u^{2} + w^{2}} + 1\right)} & 0\\\frac{u}{\sqrt{u^{2} + v^{2} + w^{2}}} & \frac{v}{\sqrt{u^{2} + v^{2} + w^{2}}} & \frac{w}{\sqrt{u^{2} + v^{2} + w^{2}}} & 0\end{matrix}\right]$$

In [5]:
data_dir_path = Path().cwd().parent.parent / "assignment"
data = load_assignment_data(data_dir_path)
# data = data.isel(t=slice(None, 1000))

data

# Same data but inversed in time.
data_inv = data.isel(t=slice(None, None, -1))
data_inv['t'] = data.t

In [6]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(data.alpha_m, data.beta_m, data.c_m)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
f_data = iekf.smooth(
    data=data.copy(),
    x_0=[150, 0, 0, 0.6065461351267598],  # 0.6021
    p_0=np.diag([1., 1., 1., 1.]),
    q=np.diag([1e-3**2, 1e-3**2, 1e-3**2, 0.**2]),
    r=np.diag([0.01**2, 0.0058**2, 0.112**2]),
    verbose=True
)

  0%|          | 0/10000 [00:00<?, ?it/s, Filtering]

  0%|          | 0/10000 [00:00<?, ?it/s, Smoothing]

In [8]:
f_data

<xarray.Dataset>
Dimensions:                 (dim_0: 4, dim_1: 4, t: 10001, x_idx: 4)
Coordinates:
  * t                       (t) float64 0.0 0.01 0.02 0.03 ... 99.98 99.99 100.0
Dimensions without coordinates: dim_0, dim_1, x_idx
Data variables: (12/32)
    alpha_m                 (t) float64 -0.004572 -0.003229 ... -0.2303 -0.2288
    beta_m                  (t) float64 0.002069 0.002634 ... 0.2543 0.2574
    V_m                     (t) float64 144.9 147.6 153.4 ... 152.1 152.1 153.9
    udot                    (t) float64 1.043e-05 6.843e-06 ... -2.153 -2.05
    vdot                    (t) float64 0.45 0.4499 0.4497 ... 24.46 24.05 23.63
    wdot                    (t) float64 2.855e-07 0.0225 ... -46.12 -43.07
    ...                      ...
    v_smoothed              (t) float64 -0.08299 -0.07849 ... 38.2 38.44
    v_smoothed_std          (t) float64 4.304e-05 4.304e-05 ... 4.283e-05
    w_smoothed              (t) float64 -0.03776 -0.03776 ... -21.1 -21.56
    w_smoothed_std          (t) float64 1.918e-05 1.918e-05 ... 1.884e-05
    C_alpha_u_smoothed      (t) float64 0.6065 0.6065 0.6065 ... 0.6065 0.6065
    C_alpha_u_smoothed_std  (t) float64 1.453e-07 1.453e-07 ... 1.453e-07

In [9]:
# Calculate Vm from the filtered and smoothed signals
f_data["vm_filtered"] = xf.sqrt(xf.square(f_data.u_filtered) + xf.square(f_data.v_filtered) + xf.square(f_data.w_filtered))
f_data["vm_smoothed"] = xf.sqrt(xf.square(f_data.u_smoothed) + xf.square(f_data.v_smoothed) + xf.square(f_data.w_smoothed))

# Calculate the estimated alpha
c_alpha_estimate = f_data.C_alpha_u_smoothed.values[0]
f_data['alpha_estimate'] = f_data.alpha_m / (1 + c_alpha_estimate)

In [10]:
fig, (ax1, ax2) = plt.subplots(2, figsize=(8, 4))
f_data.C_alpha_u_filtered.plot.line(label="C_alpha_u_filtered", ax=ax1)
f_data.C_alpha_u_smoothed.plot.line(label="C_alpha_u_smoothed", ax=ax1)
ax1.legend()

f_data.C_alpha_u_filtered_std.plot.line(label="C_alpha_u_filtered_std", ax=ax2)
f_data.C_alpha_u_smoothed_std.plot.line(label="C_alpha_u_smoothed_std", ax=ax2)
ax2.set_yscale("log")
ax2.legend()
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
plt.figure()
f_data.u_filtered.plot()
f_data.u_smoothed.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
plt.figure()
f_data.V_m.plot()
f_data.vm_filtered.plot()
f_data.vm_smoothed.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
plt.figure()
f_data.alpha_m.plot()
f_data.alpha_estimate.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
f_data.to_netcdf("data_smoothed.nc")
f_data

<xarray.Dataset>
Dimensions:                 (dim_0: 4, dim_1: 4, t: 10001, x_idx: 4)
Coordinates:
  * t                       (t) float64 0.0 0.01 0.02 0.03 ... 99.98 99.99 100.0
Dimensions without coordinates: dim_0, dim_1, x_idx
Data variables: (12/35)
    alpha_m                 (t) float64 -0.004572 -0.003229 ... -0.2303 -0.2288
    beta_m                  (t) float64 0.002069 0.002634 ... 0.2543 0.2574
    V_m                     (t) float64 144.9 147.6 153.4 ... 152.1 152.1 153.9
    udot                    (t) float64 1.043e-05 6.843e-06 ... -2.153 -2.05
    vdot                    (t) float64 0.45 0.4499 0.4497 ... 24.46 24.05 23.63
    wdot                    (t) float64 2.855e-07 0.0225 ... -46.12 -43.07
    ...                      ...
    w_smoothed_std          (t) float64 1.918e-05 1.918e-05 ... 1.884e-05
    C_alpha_u_smoothed      (t) float64 0.6065 0.6065 0.6065 ... 0.6065 0.6065
    C_alpha_u_smoothed_std  (t) float64 1.453e-07 1.453e-07 ... 1.453e-07
    vm_filtered             (t) float64 150.0 145.0 146.3 ... 155.7 155.8 155.9
    vm_smoothed             (t) float64 150.0 150.0 150.0 ... 155.7 155.8 155.9
    alpha_estimate          (t) float64 -0.002846 -0.00201 ... -0.1433 -0.1424